# Live/Design: control modes

With *pyAML* it is possible to use the same commands either:
1. live: on the real accelerator
2. design: in simulations
3. errors: in simulations with errors (and for several seeds)
4. live (with a different prefix/host): in a virtual control system (same as in point 1. for those laboratories that have a simulated control system)

Those are called "control modes" and may be used 
- at a global script level (for testing in simulations and then using in real accelerator measurements with a simple switch/comment) 
- anywhere in the script 

Using pyAML on a real accelerator

In [3]:
from pyaml.pyaml import pyaml,PyAML
from pyaml.instrument import Instrument
from pyaml.configuration.factory import Factory
import numpy as np

instantiate a pyAML accelerator

In [ ]:
ml:PyAML = pyaml('../../tests/config/EBSTune.yaml')
SR:Instrument = ml.get('sr')

Switch between live and design

In [ ]:
sr = SR.design  # simulations
# sr = SR.live # act on real accelerator

Access some magnet families defined in the configuration is simple. One can use sr.live for live control system or sr.design for the simulator part. They provide identical interface.

In [ ]:
sr.design.get_lattice() #Direct access to underlying pyAT lattice

Lattice([Marker('ID04'), Drift('DR_00J', 0.56, RApertures=array([-0.015,  0.025, -0.016,  0.016])), Dipole('S3_Septum', 0.53, 0.0, 0.0, FringeQuadEntrance=0, FringeQuadExit=0, MaxOrder=1, NumIntSteps=20, RApertures=array([-0.015,  0.025, -0.016,  0.016])), Drift('DR_01J', 0.2935, EApertures=array([0.044, 0.016])), Quadrupole('QF1J-C04', 0.403, 1.090009251235513, EApertures=array([0.044, 0.016]), FringeQuadEntrance=0, FringeQuadExit=0, NumIntSteps=20), Drift('DR_02J_H_Scraper', 0.286919240946035, EApertures=array([0.044, 0.016])), Multipole('SH1A-C04', 0.1, array([0., 0., 0.]), array([0., 0., 0.]), EApertures=array([0.025, 0.01 ]), KickAngle=array([0., 0.]), MaxOrder=2, NumIntSteps=20), Drift('DR_03J', 0.115, EApertures=array([0.044, 0.016])), Multipole('DR_K3', 0.32, array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), MaxOrder=3, NumIntSteps=20), Drift('DR_04J', 0.195), Monitor('BPM_C04-01', Offset=array([0, 0], dtype=uint8), Reading=array([0, 0], dtype=uint8), Rotation=array([0, 0], dty

In [ ]:
quadForTune = sr.get_magnets("QForTune")

tune_device = sr.get_betatron_tune_monitor('BETATRON_TUNE')

Execute actions with the magnets families (here for example a tune response matrix is evaluated)

In [ ]:
# Build tune response matrix
sr.design.get_lattice().disable_6d()
tune_design = sr.design.get_lattice().get_tune()

print(f"Tune directly from the lattice {tune_design}")
initial_tune = tune_device.tune.get()
print(f"Tune via pyAML interface for design mode {initial_tune}")
tunemat = np.zeros((len(quadForTune),2))

for idx,m in enumerate(quadForTune):
     str = m.strength.get()
     m.strength.set(str+1e-4)
     dq = tune_device.tune.get() - initial_tune
     tunemat[idx] = dq*1e4
     m.strength.set(str)

At any point in the code it is always possible to still access any of the control modes

In [ ]:
print(tune_device.tune.get())  # what ever the globally set mode whas

SR.design.get_betatron_tune_monitor('BETATRON_TUNE').tune.get()  # tune from design simulations 
# SR.live.get_betatron_tune_monitor('BETATRON_TUNE').tune.get() # tune from real accelerator